In [ ]:
# GitHub update
!git add . && git commit -m "switch to yfinance" && git push

# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
# %pip install -r requirements.txt

import numpy as np
import pandas as pd
import requests  # HTTP Requests for API interaction
import xlsxwriter
import math

So apparently, the tutorial is outdated due to IEX Cloud data provider having to change their policies over the years
and it is affecting the continuation of the course

To all those who are referring the course in 2024, following might help :


instead of IEX we can use yfinance library 


```
import yfinance as yf
tickerSymbol = 'AAPL'
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2024-05-09', end='2024-05-10')
marketCap= tickerData.info['marketCap']


price= tickerDf['Open']

and also these prices are the live market value prices.
```

In [9]:
import yfinance as yf
from datetime import datetime, timedelta

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
type(stocks)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [17]:
from secrets_0x01 import IEX_CLOUD_API_TOKEN

# there is no need for this anymore

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [ ]:
# symbol = 'AAPL'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
# print(api_url)
# data = requests.get(api_url)
# # data = requests.get(api_url).json()
# # print(data.status_code)
# print(data)

In [39]:
symbol = 'AAPL'
data = yf.Ticker(symbol)

# Fetch the quote data
quote_data = data.info

# Get the historical prices for this ticker
tickerDf = data.history(period='1d', start='2024-06-25', end='2024-06-26')
marketCap= data.info['marketCap']

price= tickerDf['Open']
# price = data.info['open']

# Print the quote data
print(quote_data)
# print(marketCap, price)

{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In add

For a more detailed example as to using `yfinance`

In [10]:
# tickerSymbol = 'AAPL'
# tickerData = yf.Ticker(tickerSymbol)

# # Get the current date and subtract one day to get a past date
# end_date = datetime.now() - timedelta(days=1)
# start_date = end_date - timedelta(days=1)

# # Format the dates as strings
# start_date_str = start_date.strftime('%Y-%m-%d')
# end_date_str = end_date.strftime('%Y-%m-%d')

# # Get the historical prices for this ticker
# tickerDf = tickerData.history(period='1d', start=start_date_str, end=end_date_str)

# # Fetch market cap
# marketCap = tickerData.info.get('marketCap', 'N/A')

# # Get the opening price
# price = tickerDf['Open'].iloc[0] if not tickerDf.empty else 'N/A'

# print(f"Market Cap: {marketCap}")
# print(f"Opening Price on {start_date_str}: {price}")

Market Cap: 3274597203968
Opening Price on 2024-06-25: 209.14999389648438


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

> **_NOTE:_**  currentPrice is similar to what latestPrice is in IEX Cloud Data.

In [41]:
price = data.info['currentPrice']
market_cap = data.info['marketCap']
print(price, market_cap/1000000000000)

214.1 3.283030900736


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [50]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
# print(final_dataframe)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [58]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24048\1829372518.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,214.1,3283030900736,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [ ]:
# Initialize the final DataFrame
final_dataframe = pd.DataFrame(columns=my_columns)

for stock in stocks['Ticker']:
    try:
        data = yf.Ticker(stock)
        stock_info = data.info
        
        # Check if the required keys are present in the info dictionary
        if 'currentPrice' in stock_info and 'marketCap' in stock_info:
            new_row = pd.DataFrame([[
                stock,
                stock_info['currentPrice'],
                stock_info['marketCap'],
                'N/A'
            ]], columns=my_columns)
            final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)
        else:
            print(f"Missing data for {stock}: 'currentPrice' or 'marketCap' not found.")
    except requests.exceptions.HTTPError as e:
        print(f"HTTPError fetching data for {stock}: {e}")
    except Exception as e:
        print(f"Error fetching data for {stock}: {e}")

print(final_dataframe)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=2rPm363PiS.


Missing data for ABC: 'currentPrice' or 'marketCap' not found.
Missing data for ALXN: 'currentPrice' or 'marketCap' not found.
Missing data for ANTM: 'currentPrice' or 'marketCap' not found.


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=2rPm363PiS.


Missing data for ATVI: 'currentPrice' or 'marketCap' not found.
Missing data for BF.B: 'currentPrice' or 'marketCap' not found.
Missing data for BLL: 'currentPrice' or 'marketCap' not found.
Missing data for BRK.B: 'currentPrice' or 'marketCap' not found.
Missing data for CERN: 'currentPrice' or 'marketCap' not found.
Missing data for COG: 'currentPrice' or 'marketCap' not found.
Missing data for CTL: 'currentPrice' or 'marketCap' not found.
Missing data for CTXS: 'currentPrice' or 'marketCap' not found.
Missing data for CXO: 'currentPrice' or 'marketCap' not found.
Missing data for DISCA: 'currentPrice' or 'marketCap' not found.
Missing data for DISCK: 'currentPrice' or 'marketCap' not found.


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DISH&crumb=2rPm363PiS.


...

Missing data for FLT: 'currentPrice' or 'marketCap' not found.
Missing data for FRC: 'currentPrice' or 'marketCap' not found.
Missing data for HFC: 'currentPrice' or 'marketCap' not found.
Missing data for INFO: 'currentPrice' or 'marketCap' not found.
Missing data for KSU: 'currentPrice' or 'marketCap' not found.
Missing data for LB: 'currentPrice' or 'marketCap' not found.
Missing data for MXIM: 'currentPrice' or 'marketCap' not found.
Missing data for MYL: 'currentPrice' or 'marketCap' not found.
Missing data for NBL: 'currentPrice' or 'marketCap' not found.
Missing data for NLOK: 'currentPrice' or 'marketCap' not found.
Missing data for NLSN: 'currentPrice' or 'marketCap' not found.
Missing data for PBCT: 'currentPrice' or 'marketCap' not found.


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PEAK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PEAK&crumb=2rPm363PiS.


Missing data for PEAK: 'currentPrice' or 'marketCap' not found.
Missing data for PKI: 'currentPrice' or 'marketCap' not found.


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=RE&crumb=2rPm363PiS.


Missing data for RE: 'currentPrice' or 'marketCap' not found.
Missing data for SIVB: 'currentPrice' or 'marketCap' not found.


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy:

## Yahoo Finance info

Here are the key fields present in the provided data:

1. **Company Information:**
   - `address1`: Company address.
   - `city`: City of the company.
   - `state`: State of the company.
   - `zip`: Zip code of the company.
   - `country`: Country of the company.
   - `phone`: Company phone number.
   - `website`: Company website URL.
   - `industry`: Industry the company operates in.
   - `industryKey`: Key for the industry.
   - `industryDisp`: Display name for the industry.
   - `sector`: Sector the company belongs to.
   - `sectorKey`: Key for the sector.
   - `sectorDisp`: Display name for the sector.
   - `longBusinessSummary`: Detailed summary of the company's business.
   - `fullTimeEmployees`: Number of full-time employees.
   - `companyOfficers`: List of company officers with details such as name, age, title, and pay.

2. **Risk Metrics:**
   - `auditRisk`: Risk associated with audit.
   - `boardRisk`: Risk associated with the board.
   - `compensationRisk`: Risk associated with compensation.
   - `shareHolderRightsRisk`: Risk associated with shareholder rights.
   - `overallRisk`: Overall risk assessment.

3. **Governance and Dates:**
   - `governanceEpochDate`: Epoch date for governance.
   - `compensationAsOfEpochDate`: Epoch date for compensation.
   - `irWebsite`: Investor relations website URL.
   - `maxAge`: Maximum age of the data.

4. **Stock Information:**
   - `priceHint`: Price hint for the stock.
   - `previousClose`: Previous closing price.
   - `open`: Opening price.
   - `dayLow`: Lowest price of the day.
   - `dayHigh`: Highest price of the day.
   - `regularMarketPreviousClose`: Previous closing price in regular market.
   - `regularMarketOpen`: Opening price in regular market.
   - `regularMarketDayLow`: Lowest price of the day in regular market.
   - `regularMarketDayHigh`: Highest price of the day in regular market.
   - `dividendRate`: Dividend rate.
   - `dividendYield`: Dividend yield.
   - `exDividendDate`: Ex-dividend date.
   - `payoutRatio`: Payout ratio.
   - `fiveYearAvgDividendYield`: Five-year average dividend yield.
   - `beta`: Beta value.
   - `trailingPE`: Trailing price-to-earnings ratio.
   - `forwardPE`: Forward price-to-earnings ratio.
   - `volume`: Trading volume.
   - `regularMarketVolume`: Trading volume in regular market.
   - `averageVolume`: Average trading volume.
   - `averageVolume10days`: Average trading volume over 10 days.
   - `averageDailyVolume10Day`: Average daily trading volume over 10 days.
   - `bid`: Bid price.
   - `ask`: Ask price.
   - `bidSize`: Bid size.
   - `askSize`: Ask size.
   - `marketCap`: Market capitalization.
   - `fiftyTwoWeekLow`: 52-week low price.
   - `fiftyTwoWeekHigh`: 52-week high price.
   - `priceToSalesTrailing12Months`: Price-to-sales ratio over the trailing 12 months.
   - `fiftyDayAverage`: 50-day moving average.
   - `twoHundredDayAverage`: 200-day moving average.
   - `trailingAnnualDividendRate`: Trailing annual dividend rate.
   - `trailingAnnualDividendYield`: Trailing annual dividend yield.
   - `currency`: Currency of the stock.
   - `enterpriseValue`: Enterprise value.
   - `profitMargins`: Profit margins.
   - `floatShares`: Number of shares available for trading.
   - `sharesOutstanding`: Number of shares outstanding.
   - `sharesShort`: Number of shares shorted.
   - `sharesShortPriorMonth`: Number of shares shorted in the prior month.
   - `sharesShortPreviousMonthDate`: Date of the previous month's short interest.
   - `dateShortInterest`: Date of the short interest.
   - `sharesPercentSharesOut`: Percentage of shares outstanding that are shorted.
   - `heldPercentInsiders`: Percentage of shares held by insiders.
   - `heldPercentInstitutions`: Percentage of shares held by institutions.
   - `shortRatio`: Short ratio.
   - `shortPercentOfFloat`: Percentage of float that is shorted.
   - `impliedSharesOutstanding`: Implied number of shares outstanding.
   - `bookValue`: Book value.
   - `priceToBook`: Price-to-book ratio.
   - `lastFiscalYearEnd`: Last fiscal year end date.
   - `nextFiscalYearEnd`: Next fiscal year end date.
   - `mostRecentQuarter`: Most recent quarter date.
   - `earningsQuarterlyGrowth`: Quarterly earnings growth.
   - `netIncomeToCommon`: Net income to common shareholders.
   - `trailingEps`: Trailing earnings per share.
   - `forwardEps`: Forward earnings per share.
   - `pegRatio`: Price/earnings to growth ratio.
   - `lastSplitFactor`: Last stock split factor.
   - `lastSplitDate`: Last stock split date.
   - `enterpriseToRevenue`: Enterprise value to revenue ratio.
   - `enterpriseToEbitda`: Enterprise value to EBITDA ratio.
   - `52WeekChange`: 52-week price change.
   - `SandP52WeekChange`: S&P 52-week price change.
   - `lastDividendValue`: Last dividend value.
   - `lastDividendDate`: Last dividend date.
   - `exchange`: Stock exchange.
   - `quoteType`: Type of quote.
   - `symbol`: Stock symbol.
   - `underlyingSymbol`: Underlying symbol.
   - `shortName`: Short name of the company.
   - `longName`: Long name of the company.
   - `firstTradeDateEpochUtc`: First trade date in epoch UTC.
   - `timeZoneFullName`: Full name of the time zone.
   - `timeZoneShortName`: Short name of the time zone.
   - `uuid`: Unique identifier.
   - `messageBoardId`: Message board ID.
   - `gmtOffSetMilliseconds`: GMT offset in milliseconds.
   - `currentPrice`: Current price.
   - `targetHighPrice`: Target high price.
   - `targetLowPrice`: Target low price.
   - `targetMeanPrice`: Target mean price.
   - `targetMedianPrice`: Target median price.
   - `recommendationMean`: Mean recommendation.
   - `recommendationKey`: Recommendation key.
   - `numberOfAnalystOpinions`: Number of analyst opinions.
   - `totalCash`: Total cash.
   - `totalCashPerShare`: Total cash per share.
   - `ebitda`: Earnings before interest, taxes, depreciation, and amortization.
   - `totalDebt`: Total debt.
   - `quickRatio`: Quick ratio.
   - `currentRatio`: Current ratio.
   - `totalRevenue`: Total revenue.
   - `debtToEquity`: Debt-to-equity ratio.
   - `revenuePerShare`: Revenue per share.
   - `returnOnAssets`: Return on assets.
   - `returnOnEquity`: Return on equity.
   - `freeCashflow`: Free cash flow.
   - `operatingCashflow`: Operating cash flow.
   - `earningsGrowth`: Earnings growth.
   - `revenueGrowth`: Revenue growth.
   - `grossMargins`: Gross margins.
   - `ebitdaMargins`: EBITDA margins.
   - `operatingMargins`: Operating margins.
   - `financialCurrency`: Financial currency.
   - `trailingPegRatio`: Trailing price/earnings to growth ratio.